## Labeled Data Collection for Classifier:
Selenium web driver opens yapi kredi FAQs site visits each page and retrieve labeled data (question and answer pairs labeled with the corresponding category name)

In [1]:
from  selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time
import pandas as pd

In [2]:
driver_path = "./chromedriver"
browser = webdriver.Chrome(executable_path=driver_path)
main_url = "https://www.yapikredi.com.tr/memnuniyetiniz-icin-buradayiz/sikca-sorulan-sorular/"

In [3]:
def getUrls(main_url):
    browser.get(main_url)
    group_URLs = []
    groupsDiv = browser.find_element_by_class_name("sss-categories")
    print(groupsDiv.text)
    all_children = groupsDiv.find_elements_by_tag_name("a")
        
    for i in range(len(all_children)):
        url = all_children[i].get_attribute("href")
        group_URLs.append(url)
    
    for each in group_URLs:
        print(each)
        
    return group_URLs

In [4]:
SubGroups = []

In [5]:
def getSubUrls(groupUrl):
    browser.get(groupUrl)
    subgroup_Urls = []
    subgroupsDiv = browser.find_element_by_class_name("gm")
    #print(subgroupsDiv.text)
    SubGroups.append(subgroupsDiv.text.split("\n"))
    sub_all_children = subgroupsDiv.find_elements_by_tag_name("li")

    subgroup_Urls =[]
    for i in range(len(sub_all_children)):
        element = sub_all_children[i].find_element_by_tag_name("a")
        url = element.get_attribute("href")
        subgroup_Urls.append(url)

    #for each in subgroup_Urls:
    #    print(each)
    
    return subgroup_Urls

In [6]:
def getNumPages(url):
    browser.get(url)
    pager = browser.find_element_by_id("ctl00_ContentPlaceHolder1_ctrlFaq_pgrFaqsBottom")
    pages = pager.find_elements_by_tag_name("li")
    num_pages = len(pages)
    #single page there is no pager 
    if num_pages == 0:
        num_pages = 1
        
    return num_pages


In [7]:
def collect_Page_Data(url, pageId):
    isNotAttached = True
    data = []
    while isNotAttached:
        try:
            browser.get(url) 
            
            singlePage = False
            try:
                pager = browser.find_element_by_id("ctl00_ContentPlaceHolder1_ctrlFaq_pgrFaqsBottom")
                page = pager.find_element_by_link_text(str(pageId))
            except NoSuchElementException:
                singlePage = True
                #print("Single page url no pager")
                
            if not singlePage:       
                page.click()
                #print("I have clicked ",page.text)
                
           
            #print("I am waiting for 6 secs")
            time.sleep(10)
            
            faqs = browser.find_elements_by_class_name("answer-item")
            for f in faqs:
                qna = "\n".join(f.text.split("\n"))
                data.append(qna)
            
            
            isNotAttached = False
        except StaleElementReferenceException:
            print("WARNING !! Exception in clickPage()")
    #print(data)
    return data
    

In [8]:
def collect_url_data(url):
    all_qnas = []
    num_pages = getNumPages(url)
    #print("Number of pages : ", num_pages)
    
    for i in range(num_pages):
        isAttached = False
        while not isAttached:
            try:
                qnas = []
                qnas = collect_Page_Data(url, i+1)
                isAttached = True
                
            except StaleElementReferenceException:
                print("Warning")             
                
        #print("attached")
        all_qnas.append(qnas)
        
    return all_qnas
    

In [ ]:
dataset = []
Urls = getUrls(main_url)

for idx_u in range(len(Urls)):
    subUrls = getSubUrls(Urls[idx_u])
    for idx_su in range(len(subUrls)):
        data = collect_url_data(subUrls[idx_su])
        tup = (idx_u, idx_su, data)
        dataset.append(tup)
        

In [ ]:
df = pd.DataFrame(dataset, columns =['GroupId', 'SubGroupId', 'Q&A'])

In [ ]:
df

In [ ]:
SubGroups = []
Urls = getUrls(main_url)

for idx_u in range(len(Urls)):
    subUrls = getSubUrls(Urls[idx_u])

In [ ]:
SubGroups

In [ ]:
import itertools

categories = list(itertools.chain.from_iterable(SubGroups))

categories

In [ ]:
df["Categories"] = categories
df

In [ ]:
#for subgroup in df["Q&A"]:
#    for page in subgroup:
#        for qna in page:
#            print(qna)
#            print()
            

In [ ]:
for sg in dataset:
    print(sg[0:2], categories[dataset.index(sg)] )

In [ ]:
#for sg in dataset:
#    for pgId in range(len(sg[2])):
#        print(sg[0:2],pgId)

In [ ]:
data = []
for sg in dataset:
    for pgId in range(len(sg[2])):
        #print(sg[0:2],pgId, sg[2][pgId])
        
        for qna in sg[2][pgId]:
            #print(qna)
            list_qna = qna.split("\n")
            list_qna.pop()
            list_qna.pop()
            processed_qna = "\n".join(list_qna)
            #print(sg[0:2],processed_qna)
            new_tup = (sg[0],sg[1],categories[dataset.index(sg)] ,processed_qna)
            data.append(new_tup)
            #print("***********")

In [ ]:
df = pd.DataFrame(data, columns =['GroupId', 'SubGroupId', 'Categories','Q&A'])
df

In [ ]:
# testUniqueness()
qnaList = df["Q&A"]
qnaSet = set(qnaList)

print(len(qnaSet),len(qnaList))

In [ ]:
#duplicates
dup = df[df.duplicated(keep = "first")]
dup

In [ ]:
df = df.drop_duplicates( keep = "first" ,inplace = False)

In [ ]:
print(df.shape)

In [ ]:
df = df.reset_index(drop=True)
df

In [ ]:
dup = df[df.duplicated(keep = "first")]
dup

In [ ]:
df.head(10)

In [ ]:
#export_csv = df.to_csv ("./labeled_qnas.csv", header=True) 
#export_xlsx = df.to_excel("./labeled_qnas.xlsx", header=True) 